## RESNET Sturctura 

Resnet turlari:
-18,34,54,152
 

![img](./data/resnet34.png "Resnet34 architecture")



In [249]:
import torch
import torchvision
from torch.nn import *


### Resnet architecture from torchvision
First things first , let's understand what are we trying to build here. Pull out resnet34 architecture from Pytorch models

In [250]:
rn34 = torchvision.models.resnet34()
print(rn34)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Basic Residual Block¶
Each basic block constitutes of 2 convolution operations. 

Each convolutional layer is followed by a batch normalization layer and a ReLU activation function. except if downsample has to be applied the 2nd conv layer's output is added to input before applying relu.

BU blockda eng asosiy ish : 
After 2 convolution operations input of those 2 convolution is added to their output.  
![img](https://raw.githubusercontent.com/jarvislabsai/blog/master/build_resnet34_pytorch/images/res_block1.png)


In [252]:
#consist of 2 conv layers  Residual block
class ResidualBlock(Module): 

    def __init__(self, in_chs, out_chs, stride=1, downsample=None) -> None:
        super().__init__()
        
        #block consist of two sequential conv block
        self.conv1 = Conv2d(in_channels=in_chs, out_channels=out_chs, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = BatchNorm2d(num_features=out_chs)
        self.conv2 = Conv2d(in_channels=out_chs, out_channels=out_chs, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = BatchNorm2d(num_features=out_chs)
        self.relu = ReLU(inplace=True)
        self.downsample = downsample        
            
    def forward(self, input): 
        identity = input #kirishdan kopiya olib qolamiz keyinchalik 2 - conv blockdan qoshish uchun
        output = self.relu(self.bn1(self.conv1(input))) #1-conv layer
        output = self.bn2(self.conv2(output)) #2-conv layer

        #identity razmerini downsample qilamiz agar output stride=2 bilan kichiklashtirilgan bo'lsa
        #Ya'ni output sizega tenglashtirib keyin qo'sha olamiz. har bir katta block 1-convda stride=2 bolgani uchn razmer outputkichiklasahdi
        if self.downsample is not None: 
            identity = self.downsample(input)
 
        output += identity #2ta convdan chiqqan natijaga undan oldingi identity ni qo'shamiz. 
        output = self.relu(output)
        return output

- Resnet34 4 ta layerdan iborat. (ya'ni 4 ta kotta block) Va har birini ichida bir nechta residual block va conv blocklar mavjud. 
- Conv_block refers to set of operations as in Convolution->BatchNorm->ReLU activation to an input

- The make_layer() funksiyasi blokni argument sifatida oladi, input va output channelarni va number of blocks qaysiki bir biriga stack bo'lishi kerak bo'lgan. Har bir layer (ya'ni kotta block)ning boshlanishida stride=2 ga teng boladi ya'ni downsampling qilinadi, lekin qolgan bir layer (katta block)dagi conv lar stride=1 xolatda qoladi. Agar downsample bo'lsa undan keyin BatchNorm bilan keladi. 

# Resnet34 

In [253]:
class ResNet34(Module):
    def __init__(self, img_channel, res_block, layer_chs, layers_num, num_classes=1000) -> None:
        super().__init__()
        """
        ResNet-34 model from scratch in pytorch as described in 'Deep Residual Learning for Image Recognition".

        :param img_channel: input image channel 
        :param res_block: Basic Block or residual block which consist of consecutively two conv layers with joining skipped input 
        :param layers_chs: Channel size of each layer (big block )
        :param layers_num: how many residual blocks have in each layer (big block)
        :num_classes: number of classes to be predicted
        """

        self.in_chs = 64

        #1-conv layer 7x7 kernelli blok 
        self.conv1 = Conv2d(img_channel, self.in_chs, kernel_size=7, stride=2, padding=3, bias=False) 
        self.bn = BatchNorm2d(self.in_chs)
        self.relu = ReLU(inplace=True)
        self.maxpool = MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.make_layer(block=res_block, out_chs=layer_chs[0], num_resblocks=layers_num[0], stride=1)
        self.layer2 = self.make_layer(block=res_block, out_chs=layer_chs[1], num_resblocks=layers_num[1], stride=2)
        self.layer3 = self.make_layer(block=res_block, out_chs=layer_chs[2], num_resblocks=layers_num[2], stride=2)
        self.layer4 = self.make_layer(block=res_block, out_chs=layer_chs[3], num_resblocks=layers_num[3], stride=2)

        self.avg_pool = AdaptiveAvgPool2d((1,1))
        self.fc = Linear(layer_chs[3], num_classes)

    def make_layer(self, block, out_chs, num_resblocks, stride): 
        #downsample - birinchi res blockdan tashqari qolganlarida birinchi convda stride=2 bolgan va bu digani
        #channel kotarilayotganda boshqa 64->128 otayotgan size kichkinalashadi
        downsample = None 
        if stride != 1 or self.in_chs != out_chs:
            downsample = Sequential(
                Conv2d(self.in_chs, out_chs, kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(out_chs))
        
        layers = [] 
        layers.append(block(self.in_chs, out_chs, stride, downsample)) 
        
        self.in_chs = out_chs
        #big blocklarni loop qilamiz. 1 chi layerni downsample orqali yozganiz uchun -1 kamroq loop qilamiz 
        for _ in range(num_resblocks-1): 
            layers.append(block(self.in_chs, out_chs, stride=1, downsample=None)) 
        
        return Sequential(*layers) #list ni torch tensor ga otkizamiz shtobi pytorch tanishi uchun 


    def forward(self, input): 
        output = self.relu(self.bn(self.conv1(input))) #224x224 -> 112 x 112 
        output = self.maxpool(output) #112 x 112 -> 56 x 56 (3,2 maxpool bn)
        output = self.layer1(output) # 56x56
        output = self.layer2(output)
        output = self.layer3(output)
        output = self.layer4(output)
        output = self.avg_pool(output)
        output = torch.flatten(output,1) #1-dim dan boshlab tekislaydi. yani 1 dimdan uyogini tekislaydi
        output = self.fc(output)


        return output
    
resnet34 = ResNet34(img_channel=3, res_block=ResidualBlock, layer_chs = [64, 128, 256, 512], layers_num = [3, 4, 6, 3], num_classes=1000)
input = torch.rand((10,3,224,224))
out = resnet34(input) 


In [254]:
resnet34

ResNet34(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64